**Projeto:** Classificação de pacientes propensos a desenvolverem doenças cardíacas.

**Etapa:** 03 - Preprocessamento

**Autor:** Dante Dantas

**Objetivo:** Transformação dos dados.

# **Principais Insights**

*Descrever aqui os principais insights*

#**Importação de pacotes**

##**Modelagem**

In [28]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
import joblib
from sklearn.metrics import precision_recall_curve, average_precision_score, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, recall_score, precision_score
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import matplotlib.pyplot as plt


In [2]:

base = pd.read_csv('/content/drive/MyDrive/Projetos/Saúde/[Classificação] Doença Cardíacas/Dados/2.TRUSTED/tb_trusted_heart_disase.csv', sep=',', encoding='utf-8')

In [3]:
# Definir features e target

features = base.iloc[:,0:11].values
target = base.iloc[:,11].values

In [4]:
# Este ColumnTransformer fará o One-Hot Encoding e passará o resto.
# Colunas [1,2,6,8,10] originais.
# São: 'sex', 'chest_pain_type', 'resting_ecg', 'exercise_angina', 'st_slope'
column_transformer = ColumnTransformer(
    transformers=[('OneHot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), [1,2,6,8,10])],
    remainder='passthrough'
)
# Ajusta e transforma TODAS as features originais (X)
transformed_features = column_transformer.fit_transform(features)

In [5]:
#  Definir e ajustar o MinMaxScaler ---
# Aplicação sobre TODAS as colunas resultantes do 'transformed_features'.

scaler = MinMaxScaler()
transformed_features_scaled = scaler.fit_transform(transformed_features)


In [6]:
#Divisão em treino/teste e treino do modelo.
x_train, x_test, y_train, y_test = train_test_split(transformed_features_scaled, target, test_size=0.3, random_state=21)

In [7]:
# Importação do modelo
from sklearn.linear_model import LogisticRegression # Exemplo
model = LogisticRegression(max_iter=1000, random_state=42) # Aumente max_iter se tiver problemas de convergência
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
# --- OBJETOS NECESSÁRIOS PARA O DEPLOY ---
# SALVE O COLUMNTRANSFORMER
joblib.dump(column_transformer, 'column_transformer.pkl')


['column_transformer.pkl']

In [ ]:
# SALVE O MINMAXSCALER
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [ ]:
# SALVE O MODELO TREINADO
joblib.dump(model, 'mdl.pkl') # Use o nome que você já estava usando

['mdl.pkl']

In [ ]:
print("Arquivos 'column_transformer.pkl', 'scaler.pkl' e 'mdl.pkl' salvos com sucesso!")


Arquivos 'column_transformer.pkl', 'scaler.pkl' e 'mdl.pkl' salvos com sucesso!




---



# Avaliação Primária

In [17]:
def score_summary(names, classifiers):
    '''
    Dada uma lista de classificadores, esta função calcula a precisão,
    Average Precision (AP) para a Curva Precision-Recall, Recall, Precision e F1-Score
    e retorna os valores em um dataframe.
    '''


    cols = ["Classifier", "Accuracy", "Average Precision (AP)", "Recall", "Precision", "F1"]
    data_table = pd.DataFrame(columns=cols)

    for name, clf in zip(names, classifiers):
        clf.fit(x_train, y_train)

        pred = clf.predict(x_test) # Previsões das classes
        accuracy = accuracy_score(y_test, pred) # Acurácia
        pred_proba = clf.predict_proba(x_test)[:, 1] # Probabilidades da classe positiva (1)

        # precision_pr, recall_pr, thresholds_pr = precision_recall_curve(y_test, pred_proba)

        # Área Sob a Curva Precision-Recall (AP)
        average_precision = average_precision_score(y_test, pred_proba)

        # MATRIZ CONFUSÃO, cm
        cm = confusion_matrix(y_test, pred) # Matriz de Confusão

        # RECALL: TP/(TP+FN)
        recall = recall_score(y_test, pred)

        # PRECISÃO: TP/(TP+FP)
        precision = precision_score(y_test, pred)

        # MEDIDA F1
        f1 = f1_score(y_test, pred)

        # Adiciona os resultados ao DataFrame
        df = pd.DataFrame([[name, accuracy * 100, average_precision, recall, precision, f1]], columns=cols)
        data_table = pd.concat([data_table, df], ignore_index=True)

    return np.round(data_table.reset_index(drop=True), 2)

In [20]:
# Avaliação Primária sem otimizações
names = ["Logistic Regression"]
classifiers = [model]

#summary_results = score_summary(names, classifiers)
#print(summary_results)

In [33]:
 score_summary(names, classifiers).sort_values(by='Recall' , ascending = False)\
.style.background_gradient(cmap='Blues')\
.bar(subset=["Average Precision (AP)"], color='#5d82c7')\
.bar(subset=["Recall"], color='#779ad5')\
.bar(subset=["Precision"], color='#90b3e3')\
.bar(subset=["F1"], color='#aacbf1')

/tmp/ipython-input-17-2866274301.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_table = pd.concat([data_table, df], ignore_index=True)


,Classifier,Accuracy,Average Precision (AP),Recall,Precision,F1
0,Logistic Regression,86.550000,0.920000,0.890000,0.870000,0.880000


# **Principais Insights**


*   **Resultado de Teste** *Durante o teste e com configuração padrão obteve esse resultado acima*
*   **Foco:** *Aumentar a Recall para minimizar os falsos negativos*